<a href="https://colab.research.google.com/github/Chandradithya8/Machine-Learning/blob/master/Hyperparameter_Tuning_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All Techniques Of Hyper Parameter Optimization

1.GridSearchCV

2.RandomizedSearchCV

3.Bayesian Optimization -Automate Hyperparameter Tuning (Hyperopt)

4.Sequential Model Based Optimization(Tuning a scikit-learn estimator with skopt)

5.Optuna- Automate Hyperparameter Tuning

6.Genetic Algorithms (TPOT Classifier)

References

1.https://github.com/fmfn/BayesianOptimization

2.https://github.com/hyperopt/hyperopt

3.https://www.jeremyjordan.me/hyperparameter-tuning/

4.https://optuna.org/

5.https://towardsdatascience.com/hyperparameters-optimization-526348bb8e2d(By 
Pier Paolo Ippolito )

6.https://scikit-optimize.github.io/stable/auto_examples/hyperparameter-optimization.html

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
df=pd.read_csv("/content/diabetes.csv.txt")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
import numpy as np
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Insulin']=np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35,30.5,33.6,0.627,50,1
1,1,85.0,66,29,30.5,26.6,0.351,31,0
2,8,183.0,64,0,30.5,23.3,0.672,32,1
3,1,89.0,66,23,94.0,28.1,0.167,21,0
4,0,137.0,40,35,168.0,43.1,2.288,33,1


In [ ]:
x=df.drop('Outcome',axis=1)
y=df['Outcome']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=33)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=10).fit(x_train,y_train)
y_pred=rf.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

[[125  44]
 [ 21  41]]
0.7186147186147186
              precision    recall  f1-score   support

           0       0.86      0.74      0.79       169
           1       0.48      0.66      0.56        62

    accuracy                           0.72       231
   macro avg       0.67      0.70      0.68       231
weighted avg       0.76      0.72      0.73       231



The main parameters used by a Random Forest Classifier are:

criterion = the function used to evaluate the quality of a split.

max_depth = maximum number of levels allowed in each tree.

max_features = maximum number of features considered when splitting a node.

min_samples_leaf = minimum number of samples which can be stored in a tree leaf.

min_samples_split = minimum number of samples necessary in a node to cause node splitting.

n_estimators = number of trees in the ensamble.

In [ ]:
rf=RandomForestClassifier(n_estimators=300, criterion='entropy', max_depth=10, min_samples_split=33, min_samples_leaf=5,).fit(x_train,y_train)
y_pred=rf.predict(x_test)
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

[[132  43]
 [ 14  42]]
0.7532467532467533
              precision    recall  f1-score   support

           0       0.90      0.75      0.82       175
           1       0.49      0.75      0.60        56

    accuracy                           0.75       231
   macro avg       0.70      0.75      0.71       231
weighted avg       0.80      0.75      0.77       231



Randomized search *CV*

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators=[int(x) for x in np.linspace(200,2000,10)]
criterion=['gini','entropy']
max_depth=[int(y) for y in np.linspace(10,50,20) ]
min_samples_split=[int(z) for z in np.linspace(10,100,25)]
min_samples_leaf=[int(p) for p in np.linspace(1,20,5)]                                             
max_features=['auto','log2','sqrt']
grid={
    'criterion':criterion,
    'n_estimators':n_estimators,
    'max_depth':max_depth,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'max_features':max_features
}
print(grid)

{'criterion': ['gini', 'entropy'], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_depth': [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 31, 33, 35, 37, 39, 41, 43, 45, 47, 50], 'min_samples_split': [10, 13, 17, 21, 25, 28, 32, 36, 40, 43, 47, 51, 55, 58, 62, 66, 70, 73, 77, 81, 85, 88, 92, 96, 100], 'min_samples_leaf': [1, 5, 10, 15, 20], 'max_features': ['auto', 'log2', 'sqrt']}


In [ ]:
rs=RandomizedSearchCV(RandomForestClassifier(),grid,cv=5,n_iter=100,verbose=5,n_jobs=-1,random_state=100)
rs.fit(x_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 11.0min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [ ]:
rs.best_params_

{'criterion': 'entropy',
 'max_depth': 24,
 'max_features': 'auto',
 'min_samples_leaf': 5,
 'min_samples_split': 43,
 'n_estimators': 200}

In [ ]:
model=rs.best_estimator_

In [ ]:
y_pred=model.predict(x_test)
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

[[134  42]
 [ 12  43]]
0.7662337662337663
              precision    recall  f1-score   support

           0       0.92      0.76      0.83       176
           1       0.51      0.78      0.61        55

    accuracy                           0.77       231
   macro avg       0.71      0.77      0.72       231
weighted avg       0.82      0.77      0.78       231



GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
params_grid={
    'criterion':[rs.best_params_['criterion']],
    'n_estimators':[rs.best_params_['n_estimators'],rs.best_params_['n_estimators']-200,rs.best_params_['n_estimators']+200,rs.best_params_['n_estimators']+100,rs.best_params_['n_estimators']-100],
    'max_depth':[rs.best_params_['max_depth']],
    'min_samples_split':[rs.best_params_['min_samples_split'],rs.best_params_['min_samples_split']-2,rs.best_params_['min_samples_split']-1,rs.best_params_['min_samples_split']+1,rs.best_params_['min_samples_split']+2,rs.best_params_['min_samples_split']-3,rs.best_params_['min_samples_split']+3],
    'min_samples_leaf':[rs.best_params_['min_samples_leaf'],rs.best_params_['min_samples_leaf']-5,rs.best_params_['min_samples_leaf']+5,rs.best_params_['min_samples_leaf']-3,rs.best_params_['min_samples_leaf']-2,rs.best_params_['min_samples_leaf']-1],
    'max_features':['auto']
    }

In [ ]:
gs=GridSearchCV(RandomForestClassifier(),params_grid,cv=3,n_jobs=-1,verbose=5)
gs.fit(x_train,y_train)

Fitting 3 folds for each of 210 candidates, totalling 630 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-1)]: Done 342 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 630 out of 630 | elapsed:  2.5min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [ ]:
gs.best_params_

{'criterion': 'entropy',
 'max_depth': 24,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 43,
 'n_estimators': 200}

In [ ]:
model1=gs.best_estimator_

In [ ]:
y_pred=model1.predict(x_test)
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

[[128  41]
 [ 18  44]]
0.7445887445887446
              precision    recall  f1-score   support

           0       0.88      0.76      0.81       169
           1       0.52      0.71      0.60        62

    accuracy                           0.74       231
   macro avg       0.70      0.73      0.71       231
weighted avg       0.78      0.74      0.76       231



Genetic Algorithms

Genetic Algorithms tries to apply natural selection mechanisms to Machine Learning contexts.

Let's immagine we create a population of N Machine Learning models with some predifined Hyperparameters. We can then calculate the accuracy of each model and decide to keep just half of the models (the ones that performs best). We can now generate some offsprings having similar Hyperparameters to the ones of the best models so that go get again a population of N models. At this point we can again caltulate the accuracy of each model and repeate the cycle for a defined number of generations. In this way, just the best models will survive at the end of the process.

In [ ]:
import pandas as pd
df=pd.read_csv("/content/diabetes.csv.txt")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
import numpy as np
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35,0,33.6,0.627,50,1
1,1,85.0,66,29,0,26.6,0.351,31,0
2,8,183.0,64,0,0,23.3,0.672,32,1
3,1,89.0,66,23,94,28.1,0.167,21,0
4,0,137.0,40,35,168,43.1,2.288,33,1


In [ ]:
x=df.drop('Outcome',axis=1)
y=df['Outcome']

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt','log2']
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
min_samples_split = [2, 5, 10,14]
min_samples_leaf = [1, 2, 4,6,8]
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(param)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [ ]:
pip install tpot

     |████████████████████████████████| 92kB 2.4MB/s 
     |████████████████████████████████| 163kB 8.2MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=9e8200d30a041693bc053f72ed2da92f14d6b5874b08233c8dff51109835ff39
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7)

In [ ]:
from tpot import TPOTClassifier
tpot_classifier=TPOTClassifier(generations=5, population_size=24,
                               offspring_size=12,verbosity= 2, early_stop= 12,
                               scoring='accuracy',config_dict={
                                   'sklearn.ensemble.RandomForestClassifier':param},
                                   cv=3
                               )
tpot_classifier.fit(x_train,y_train)

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7f6e640afd08>
Traceback (most recent call last):
  File "/usr/lib/python3.6/_weakrefset.py", line 38, in _remove
    def _remove(item, selfref=ref(self)):
stopit.utils.TimeoutException



Generation 1 - Current best internal CV score: 0.7728119180633147
Generation 2 - Current best internal CV score: 0.7728119180633147
Generation 3 - Current best internal CV score: 0.7746741154562384
Generation 4 - Current best internal CV score: 0.7746741154562384

Best pipeline: RandomForestClassifier(RandomForestClassifier(CombineDFs(input_matrix, input_matrix), criterion=gini, max_depth=560, max_features=sqrt, min_samples_leaf=8, min_samples_split=10, n_estimators=1600), criterion=gini, max_depth=340, max_features=sqrt, min_samples_leaf=4, min_samples_split=14, n_estimators=1800)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                 

In [ ]:
tpot_classifier.score(x_test,y_test)

0.7748917748917749

**Automated Hyperparameter Tuning**

Automated Hyperparameter Tuning can be done by using techniques such as

Bayesian Optimization

Gradient Descent

Evolutionary Algorithms

**Bayesian Optimization**

Bayesian optimization uses probability to find the minimum of a function. The final aim is to find the input value to a function which can gives us the lowest possible output value.It usually performs better than random,grid and manual search providing better performance in the testing phase and reduced optimization time. In Hyperopt, Bayesian Optimization can be implemented giving 3 three main parameters to the function fmin.

Objective Function = defines the loss function to minimize.

Domain Space = defines the range of input values to test (in Bayesian Optimization this space creates a probability distribution for each of the used Hyperparameters).

Optimization Algorithm = defines the search algorithm to use to select the best input values to use in each new iteration.

https://towardsdatascience.com/hyperparameter-optimization-in-python-part-2-hyperopt-5f661db91324


In [1]:
import pandas as pd
df=pd.read_csv("/content/diabetes.csv.txt")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
import numpy as np
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35,0,33.6,0.627,50,1
1,1,85.0,66,29,0,26.6,0.351,31,0
2,8,183.0,64,0,0,23.3,0.672,32,1
3,1,89.0,66,23,94,28.1,0.167,21,0
4,0,137.0,40,35,168,43.1,2.288,33,1


In [4]:
x=df.drop('Outcome',axis=1)
y=df['Outcome']

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7)

In [7]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [10]:
space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.quniform('max_depth', 10, 1200, 10),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [10, 50, 300, 750, 1200,1300,1500])
    }
space

{'criterion': <hyperopt.pyll.base.Apply at 0x7f57d5e62a58>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x7f57d5e62c50>,
 'max_features': <hyperopt.pyll.base.Apply at 0x7f57d5e62d68>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x7f57d5e62f98>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x7f57d5e68128>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x7f57d5e68278>}

In [13]:
from sklearn.model_selection import cross_val_score
def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'], 
                                 )
    
    accuracy = cross_val_score(model, x_train, y_train, cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }

In [14]:
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 80,
            trials= trials)
best

100%|██████████| 80/80 [09:10<00:00,  6.88s/it, best loss: -0.7784181377639322]


{'criterion': 1,
 'max_depth': 650.0,
 'max_features': 2,
 'min_samples_leaf': 0.001369131811931129,
 'min_samples_split': 0.00964724336125422,
 'n_estimators': 6}

In [15]:
crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 300, 3: 750, 4: 1200,5:1300,6:1500}


print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

gini
log2
1500


In [16]:
best['min_samples_leaf']


0.001369131811931129

In [19]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], max_depth = best['max_depth'], 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = est[best['n_estimators']]).fit(x_train,y_train)
predictionforest = trainedforest.predict(x_test)
print(confusion_matrix(y_test,predictionforest))
print(accuracy_score(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc5 = accuracy_score(y_test,predictionforest)

[[130  18]
 [ 37  46]]
0.7619047619047619
              precision    recall  f1-score   support

           0       0.78      0.88      0.83       148
           1       0.72      0.55      0.63        83

    accuracy                           0.76       231
   macro avg       0.75      0.72      0.73       231
weighted avg       0.76      0.76      0.75       231



**Optimize hyperparameters of the model using Optuna**

The hyperparameters of the above algorithm are n_estimators and max_depth for which we can try different values to see if the model accuracy can be improved. The objective function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [21]:
pip install optuna

     |████████████████████████████████| 256kB 4.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163kB 12.8MB/s 
     |████████████████████████████████| 81kB 6.5MB/s 
     |████████████████████████████████| 81kB 4.0MB/s 
     |████████████████████████████████| 112kB 17.2MB/s 
     |████████████████████████████████| 143kB 16.1MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
  Created wheel for optuna: filename=optuna-2.2.0-cp36-none-any.whl size=340928 sha256=2368c5732194b438778c518cbefc877bec0b639904c8e516a34351a51400bfd5
  Stored in directory: /root/.cache/pip/wheels/a6/27/26/3ef57a8fb2c8bceef3f4ece73483cec1895e06eae442d34a41
Successfully built optuna
  Created wheel for pyperclip: filename=pyperclip-1.8.1-cp36-none-any.whl size=11119 sha256=752b72f158c2c6a8482ee6a253bddd555e1a6384d0d15ef90c01e0fa252905c2
  Stored in directory: /root/.cache/pip/wheels/

In [24]:
import optuna
import sklearn.svm
def objective(trial):

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
        max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf,x_train,y_train, n_jobs=-1, cv=3).mean()

In [25]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2020-10-21 12:59:44,346] A new study created in memory with name: no-name-3c55cf24-ed2b-4841-b6c8-71cb6356be44
[I 2020-10-21 12:59:45,879] Trial 0 finished with value: 0.6554934823091249 and parameters: {'classifier': 'SVC', 'svc_c': 18237218.307229165}. Best is trial 0 with value: 0.6554934823091249.
[I 2020-10-21 12:59:48,671] Trial 1 finished with value: 0.7802607076350093 and parameters: {'classifier': 'RandomForest', 'n_estimators': 610, 'max_depth': 13.075954439981574}. Best is trial 1 with value: 0.7802607076350093.
[I 2020-10-21 12:59:48,716] Trial 2 finished with value: 0.6554934823091249 and parameters: {'classifier': 'SVC', 'svc_c': 1.280093438654357e-07}. Best is trial 1 with value: 0.7802607076350093.
[I 2020-10-21 12:59:56,049] Trial 3 finished with value: 0.7821229050279329 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1680, 'max_depth': 22.497915785724857}. Best is trial 3 with value: 0.7821229050279329.
[I 2020-10-21 13:00:02,805] Trial 4 finished w

Accuracy: 0.7895716945996275
Best hyperparameters: {'classifier': 'RandomForest', 'n_estimators': 1420, 'max_depth': 26.91469376384657}


In [26]:
trial

FrozenTrial(number=30, value=0.7895716945996275, datetime_start=datetime.datetime(2020, 10, 21, 13, 1, 35, 715282), datetime_complete=datetime.datetime(2020, 10, 21, 13, 1, 41, 816335), params={'classifier': 'RandomForest', 'n_estimators': 1420, 'max_depth': 26.91469376384657}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest', 'SVC')), 'n_estimators': IntUniformDistribution(high=2000, low=200, step=10), 'max_depth': LogUniformDistribution(high=100, low=10)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=30, state=TrialState.COMPLETE)

In [27]:
study.best_params

{'classifier': 'RandomForest',
 'max_depth': 26.91469376384657,
 'n_estimators': 1420}

In [29]:
rf=RandomForestClassifier(n_estimators=330,max_depth=30)
rf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=330,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [30]:
y_pred=rf.predict(x_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[133  15]
 [ 36  47]]
0.7792207792207793
              precision    recall  f1-score   support

           0       0.79      0.90      0.84       148
           1       0.76      0.57      0.65        83

    accuracy                           0.78       231
   macro avg       0.77      0.73      0.74       231
weighted avg       0.78      0.78      0.77       231

